In [2]:
%%javascript
 IPython.OutputArea.prototype._should_scroll = function(lines) {
     return false;
 }

<IPython.core.display.Javascript object>

In [3]:
# Imports
# pip installs: geojson, pandas, ipyleaflet, svgpath2mpl, bokeh, openpyxl

import geojson, re
import pandas as pd
import numpy as np
from ipywidgets import widgets, Layout, ToggleButtonsStyle
from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoJSON, Polygon, Popup
from datetime import datetime
import xml.etree.ElementTree as etree
from svgpath2mpl import parse_path
from bokeh.io import output_notebook
from bokeh.io import show, curdoc
from bokeh.plotting import figure
from bokeh import events
from bokeh.models import CustomJS, Div, TapTool, ColumnDataSource, MultiPolygons, Plot, LinearAxis, Grid
from bokeh.layouts import column as bkCol
from bokeh.layouts import row as bkRow

In [4]:
output_notebook()

Loading BokehJS ...

In [4]:
# Constituency classes

class CONSTITUENCY():
    
    const_list = {}
    previous_list = {}
    
    def __init__(self,data):
        self.name = data['Name'] # string
        self.region = data['Region'] # string
        self.county = data['County'] # string
        self.created = get_election_date_from_string(data['Created']) # datetime
        self.predecessors1 = data['Original Predecessors'] # string
        self.abolished = self.split_dates(data['Abolished']) # list
        self.successors = data['Successors'] # string
        self.recreated = self.split_dates(data['Re-created']) # list
        self.predecessors2 = data['Predecessors'] # string
        self.previous = data['Previous Name'] # string
        self.changed = self.split_dates(data['Changed']) # datetime
        self.tripleMP = self.split_dates(data['3 MPs']) # datetime
        self.doubleMP = self.split_dates(data['2 MPs']) # datetime
        self.singleMP = self.split_dates(data['1 MP']) # datetime
        self.election_list = {} # dictionary
        CONSTITUENCY.const_list.update({self.name:self})
        if self.previous != '':
            for name in self.previous.split('|'):
                CONSTITUENCY.previous_list.update({name:self})
    
    def split_dates(self,date_string):
        
        if date_string == '':
            return []
        
        if isinstance(date_string,int):
            date_string = str(date_string)
        
        date_list = date_string.split('|')
        
        date_list = [get_election_date_from_string(x) for x in date_list]
        
        return date_list
    
    def update_results(self,date_string,res_list):
        
        date = get_election_date_from_string(date_string)
        
        if not self.existed(date):
            print("Results for when const didn't exist",self.name,date_string)
            raise Exception
        
        seats = self.get_seats(date)
        
        if date_string[-1] == 'B':
            if date_string[-2] != 'B':
                if seats == 2 and not self._has_ex_mp(res_list):
                    print("No candidate for by-election with two seats",self.name,date_string)
                    raise Exception
                seats = 1
            res = CONST_RESULT(res_list,seats,True)
            self.election_list.update({date_string:res})            
        else:
            res = CONST_RESULT(res_list,seats,False)
            self.election_list.update({date_string:res})
            
    def get_seats(self,date):
        
        if self.singleMP != [] and date >= self.singleMP[0] and date < self.singleMP[1]:
            return 1
        elif self.doubleMP != [] and date >= self.doubleMP[0] and date < self.doubleMP[1]:
            return 2
        elif self.tripleMP != [] and date >= self.tripleMP[0] and date < self.tripleMP[1]:
            return 3
        else:
            return 1
    
    def _has_ex_mp(self,res_list):
        
        if len(res_list) == 1:
            
            if res_list[0]['Notes'][0] != '[':
                return False
            else:
                return True
        
        for res in res_list:
            if res['Party'] == 'Turnout':
                if res['Notes'][0] != '[':
                    return False
                else:
                    return True
    
    def existed(self,date):
        
        if date < self.created:
            return False
        
        if len(self.abolished) > 0 and len(self.recreated) == 0:
            
            if date > self.abolished[0]:
                return False
            
        elif len(self.abolished) > 0 and len(self.recreated) > 0:
            
            for i in range(0,len(self.abolished)):
                if i+1 > len(self.recreated) and date >= self.abolished[i]:
                    return False
                if date >= self.abolished[i] and date < self.recreated[i]:
                    return False
        
        return True
        
    def get_mps(self,date):
        
        elections = list(self.election_list.keys())
        elections = {get_election_date_from_string(x):x for x in elections}
        last_election = ''
        for election in sorted(elections.keys()):
            if date < election:
                if last_election == '':
                    if self.existed(date):
                        mp_list = "Constituency existed but results not available"
                    else:
                        mp_list = "Constituency not in existance"
                    break
                if self.get_seats(get_election_date_from_string(elections[last_election])) == 1:
                    mp_list = self.election_list[elections[last_election]].winner1
                    break
                else:
                    w1 = self.election_list[elections[last_election]].winner1
                    w2 = self.election_list[elections[last_election]].winner2
                    if w2 == '':
                        lw1 = self.election_list[elections[last_last_election]].winner1
                        lw2 = self.election_list[elections[last_last_election]].winner2
                        notes = self.election_list[elections[last_election]].notes
                        ex_mp = notes.split(']')[0].replace('[','')
                        if lw1.split(' ')[-1] == ex_mp:
                            w2 = lw2
                        else:
                            w2 = lw1
                    mp_list = w1 + ", " + w2
                    break
            if last_election == '':
                last_last_election = ''
                last_election = election
            else:
                last_last_election = last_election
                last_election = election
        
        return mp_list
    
    def check_elections(self):
        
        pass
    
class CONST_RESULT():
    
    def __init__(self,results,seats,byelection):
        self.results = results
        self.winner1 = ''
        self.party1 = ''
        self.winner2 = ''
        self.party2 = ''
        self.winnerN = 0
        self.winner2N = 0
        self.winnerP = 0
        self.winner2P = 0
        self.majority = 0
        self.unopposed = False
        self.turnoutN = 0
        self.turnoutP = 0
        self.notes = ''
        self.seats = seats
        self.overturned = False
        self.byelection = byelection
        self.process_results()
        
    def process_results(self):
        
        df_list = []
        
        if self.seats == 1:
        
            if len(self.results) == 1:
                self.winner1 = self.results[0]['Candidate']
                self.party1 = self.results[0]['Party']
                self.notes = self.results[0]['Notes']
                self.unopposed = True
                self.results = pd.DataFrame(self.results)
            else:
                for res in self.results:
                    if res['Party'] == 'Turnout':
                        self.turnoutN = res['Votes']
                        self.turnoutP = res['Percent']
                        self.notes = res['Notes']
                    else:
                        if isinstance(res['Votes'],str) and '*' in res['Votes']:
                            res['Votes'] = int(res['Votes'].replace('*',''))
                            self.overturned = True
                        df_list.append(res)
                
                self.results = pd.DataFrame(df_list)
                self.results.sort_values('Votes',ascending=False)
                self.results.reset_index(inplace=True)
                
                if isinstance(self.results['Votes'][0],str) and self.overturned:
                    self.winner1 = self.results['Candidate'][1]
                    self.party1 = self.results['Party'][1]
                    self.winner1N = self.results['Votes'][1]
                    self.winner1P = self.results['Percent'][1]
                    if len(self.results['Votes']) > 2:
                        self.majority = self.results['Votes'][1] - self.results['Votes'][2]
                else:
                    self.winner1 = self.results['Candidate'][0]
                    self.party1 = self.results['Party'][0]
                    self.winner1N = self.results['Votes'][0]
                    self.winner1P = self.results['Percent'][0]
                    self.majority = self.results['Votes'][0] - self.results['Votes'][1]
                
        else:
            
            if len(self.results) == 2:
                self.winner1 = self.results[0]['Candidate']
                self.party1 = self.results[0]['Party']
                self.winner2 = self.results[1]['Candidate']
                self.party2 = self.results[1]['Party']
                self.notes = self.results[1]['Notes']
                self.unopposed = True
                self.results = pd.DataFrame(self.results)
            else:
                for res in self.results:
                    if res['Party'] == 'Turnout':
                        self.turnoutN = res['Votes']
                        self.turnoutP = res['Percent']
                        self.notes = res['Notes']
                    else:
                        if isinstance(res['Votes'],str) and '*' in res['Votes']:
                            res['Votes'] = int(res['Votes'].replace('*',''))
                            self.overturned = True
                            ot_candidate = res['Candidate']
                        df_list.append(res)                
                self.results = pd.DataFrame(df_list)
                self.results.sort_values('Votes',ascending=False)
                self.results.reset_index(inplace=True)
                numbers = [0,1]
                if self.overturned and ot_candidate == self.results['Candidate'][0]:
                    numbers = [1,2]
                elif self.overturned and ot_candidate == self.results['Candidate'][1]:
                    numbers = [0,2]
                self.winner1 = self.results['Candidate'][numbers[0]]
                self.party1 = self.results['Party'][numbers[0]]
                self.winner1N = self.results['Votes'][numbers[0]]
                self.winner1P = self.results['Percent'][numbers[0]]
                self.winner2 = self.results['Candidate'][numbers[1]]
                self.party2 = self.results['Party'][numbers[1]]
                self.winner2N = self.results['Votes'][numbers[1]]
                self.winner2P = self.results['Percent'][numbers[1]]

def get_election_date_from_string(date_string):
    
    try:
        if date_string == '':
            return ''

        if not isinstance(date_string,str):
            if date_string < 1830 or date_string == 1861: # Birkenhead created between elections
                return datetime.strptime(str(date_string),'%Y')
            date_string = str(int(date_string))

        byelection = False

        if date_string[-1] == 'B':
            date_string = date_string[:-1]
            byelection = True
            if date_string[-1] == 'B':
                date_string = date_string[:-1]

        if byelection:

            if len(date_string) == 4:
                return datetime.strptime(date_string,'%Y')
            elif len(date_string) == 8:
                return datetime.strptime(date_string,'%Y %b')
            else:
                return datetime.strptime(date_string,'%Y %d%b')
        else:

            return ELECTION.elect_list[date_string].start_date
    
    except:
        print(date_string)
        raise Exception


In [5]:
# Other classes

class ELECTION():
    
    elect_list = {}
    year_list = set()
    
    def __init__(self,data):
        self.year = str(data['Year'])
        self.start_date, self.end_date = self.parse_dates(data['Date'])
        self.largest_party = data['Largest Party']
        self.pm = data['Prime Minister']
        self.turnout = data['Turnout']
        self.const_list = []
        self.result = {}
        self.result_df = ''
        self.total_votes = 0
        ELECTION.elect_list.update({self.year:self})
        ELECTION.year_list.add(self.year[:4])
        
    def parse_dates(self,date_string):
        
        if not isinstance(date_string,datetime):
            date_string = date_string.split('-')
            return datetime.strptime(date_string[0],'%d/%m/%Y'), datetime.strptime(date_string[1],'%d/%m/%Y')
        else:
            return date_string, ''

    def get_consts(self):
        
        self.const_list = []
        
        for const in CONSTITUENCY.const_list.values():
            if self.year in const.election_list.keys():
                self.const_list.append(const)
        
    def get_results(self):
                
        if self.const_list == []:
            self.get_consts()
        
        for const in self.const_list:
            
            res = const.election_list[self.year]
            
            for row in res.results.index:
                party = self.check_alliance(res.results['Party'][row])
                #party = res.results['Party'][row]
                if party not in self.result.keys():
                    self.result[party] = {'Votes':0,'Seats':0,'Percent':0}
                if res.results['Votes'][row] != 'Unopposed':
                    try:
                        self.result[party]['Votes'] += res.results['Votes'][row]
                        self.total_votes += res.results['Votes'][row]
                    except:
                        raise Exception(const.name,self.year)
                if res.results['Party'][row] == res.party1:
                    self.result[party]['Seats'] += 1
                if res.results['Party'][row] == res.party2:
                    self.result[party]['Seats'] += 1
        
        for party in self.result.keys():
            self.result[party]['Percent'] = self.result[party]['Votes'] / self.total_votes * 100
            
        sum_dict = {'Party':[],'Seats':[],'Votes':[],'Percent':[]}
        
        for party in self.result.keys():
            sum_dict['Party'].append(party)
            sum_dict['Seats'].append(self.result[party]['Seats'])
            sum_dict['Votes'].append(self.result[party]['Votes'])
            sum_dict['Percent'].append(self.result[party]['Percent'])
        
        df = pd.DataFrame(sum_dict)
        df.sort_values(by='Party',inplace=True)
        df.sort_values(by=['Seats','Votes'],inplace=True,ascending=False)
        self.result_df = df
    
    def check_alliance(self,party):
        
        if party in PARTY.party_list.keys() and PARTY.party_list[party].coalition != '':
            return PARTY.party_list[party].coalition
        
        return party

class PARTY():
    
    party_list = {}
    
    def __init__(self,name,colour,colour_scale,coalition=False):
        self.name = name
        self.colour = colour
        self.colour_scale = colour_scale.split(',')
        self.coalition = coalition
        PARTY.party_list[self.name] = self

def check_results(df):
    
    for row in df.index:
        const = df['Constituency'][row]
        year = df['Year'][row]
        if year[-1] == 'B':
            if year[:4] in ELECTION.year_list and len(year) < 9:
                print("By-election in general election year but no month given")
                print(row,year,const)
                raise Exception
        else:
            if year not in ELECTION.elect_list.keys():
                print("General election not in election list")
                print(row,year,const)
                raise Exception

def process_consts(df):
    
    for row in df.index:
        CONSTITUENCY(df.iloc[row].to_dict())
        
def process_elections(df):
    
    for row in df.index:
        ELECTION(df.iloc[row].to_dict())
        
def process_results(df):
    
    log = widgets.HTML("")
    display(log)
    
    df['Notes'].fillna('',inplace=True)
    
    const_elect = {}
    
    for row in df.index:
        const = df['Constituency'][row]
        year = str(df['Year'][row])
        if const not in const_elect.keys():
            const_elect[const] = {}
        if year not in const_elect[const].keys():
            const_elect[const][year] = []
        if df['Votes'][row] != 'Unopposed':
            if not isinstance(df['Votes'][row],int):
                try:
                    int(df['Votes'][row].replace(",","").replace("*",""))
                except:
                    print("Vote not integer",const,year)
                    raise Exception
            if not isinstance(df['Percent'][row],float) and not isinstance(df['Percent'][row],int):
                try:
                    float(df['Percent'][row].replace(" (est)",""))
                except:
                    print("Percent not float",const,year)
                    raise Exception     
        elect_dict = {
                      'Party': df['Party'][row],
                      'Candidate': df['Candidate'][row],
                      'Votes': df['Votes'][row],
                      'Percent': df['Percent'][row],
                      'Notes': df['Notes'][row]
                     }
        const_elect[const][year].append(elect_dict)
    
    for const in const_elect.keys():
        const_obj = CONSTITUENCY.const_list[const]
        for year in const_elect[const].keys():
            if year[-1] == 'B':
                byelection = True
                if year[-2] == 'B':
                    byyear = year[:-2]
                else:
                    byyear = year[:-1]
            else:
                byelection = False
            if byelection and byyear in ELECTION.year_list and len(byyear) == 4:
                print("Byelection in gen year without month",const,year)
                raise Exception
            if not byelection and year not in ELECTION.elect_list.keys():
                print("Not election year",const,year)
                raise Exception
            turnout = False
            notes = False
            for res in const_elect[const][year]:
                if res['Party'] == 'Turnout' and turnout:
                    print("Multiple Turnout",const,year)
                    raise Exception
                elif res['Party'] == 'Turnout':
                    turnout = True
                if res['Notes'] != '':
                    notes = True
            vote_set = list(set([x['Votes'] for x in const_elect[const][year]]))
            if len(vote_set) == 1 and vote_set[0] == 'Unopposed':
                turnout = True
            
            if not turnout:
                print("Missing turnout",const,year)
                raise Exception
            if byelection and not notes:
                print("Byelection no notes",const,year)
                raise Exception
            
            log.value = str(const) + " " + str(year)
            const_obj.update_results(year,const_elect[const][year])

def process_parties(df):
    
    df.fillna('',axis=1,inplace=True)
    for row in df.index:
        PARTY(df.Party[row],df.Colour[row],df['Colour Scale'][row],df['Main Party'][row])


In [6]:
election_df = pd.read_excel("UK_Elections.xlsm",sheet_name="Elections")
process_elections(election_df)

const_sum_df = pd.read_excel("UK_Elections.xlsm",sheet_name="Const_sum")
const_sum_df = const_sum_df.fillna('')

for row in const_sum_df.index:
    CONSTITUENCY(const_sum_df.loc[row])
    
const_df = pd.read_excel("UK_Elections.xlsm",sheet_name="Const_full")
process_results(const_df)

party_df = pd.read_excel("UK_Elections.xlsm",sheet_name="Parties")
process_parties(party_df)

for year in ELECTION.elect_list.keys():
    ELECTION.elect_list[year].get_results()


HTML(value='')

In [7]:
# List of colours for counties
county_colours = {"Essex":"orange",
"Bedfordshire":"green",
"Norfolk":"black",
"Hertfordshire":"pink",
"Suffolk":"blue",
"Cambridgeshire":"purple",
"Derbyshire":"blue",
"Nottinghamshire":"green",
"Lincolnshire":"pink",
"Leicestershire":"yellow",
"Northamptonshire":"lightgreen",
"London":"red",
"Northumberland":"red",
"Durham":"blue",
"Tyne and Wear":"pink",
"Cleveland":"green",
"Greater Manchester":"green",
"Cumbria":"orange",
"Merseyside":"lightgreen",
"Lancashire":"purple",
"Cheshire":"yellow",
"Northern Ireland":"red",
"Scotland":"green",
"Hampshire":"purple",
"West Sussex":"yellow",
"Kent":"blue",
"Buckinghamshire":"black",
"Oxfordshire":"blue",
"East Sussex":"green",
"Berkshire":"lightgreen",
"Surrey":"orange",
"Isle of Wight":"red",
"Wiltshire":"blue",
"Avon":"red",
"Dorset":"green",
"Somerset":"yellow",
"Cornwall":"blue",
"Devon":"orange",
"Gloucestershire":"purple",
"Wales":"pink",
"West Midlands":"red",
"Worcestershire":"green",
"Staffordshire":"lightblue",
"Herefordshire":"yellow",
"Warwickshire":"orange",
"Shropshire":"black",
"South Yorkshire":"orange",
"West Yorkshire":"yellow",
"Humberside":"purple",
"North Yorkshire":"red",}


In [5]:
# Load data
with open('Data/GB/2010-2019_Consts.geojson') as f:
    gj_2010_2019 = geojson.load(f)

with open('Data/GB/2005_Consts.geojson') as f:
    gj_2005 = geojson.load(f)
    
# with open('Data/US/congress20205m.geojson') as f:
#     us_congress_gj = geojson.load(f)

# with open('Data/US/congress20205m.geojson') as f:
#     us_states_gj = geojson.load(f)

# with open('Data/Can/canED.geojson') as f:
#     can_gj = geojson.load(f)

map_dict = {
            'UK':{'elections':{'2019':gj_2010_2019,
                               '2017':gj_2010_2019,
                               '2015':gj_2010_2019,
                               '2010':gj_2010_2019,
                               '2005':gj_2005,
                               '2001':'1997',
                               '1997':'1997',
                               '1992':'1992',
                               '1987':'1992',
                               '1983':'1992',}},
#             'US':{'elections':{'2020':us_congress_gj}},
#             'Canada':{'elections':{'2021':can_gj}}
           }


# us_hex_df = pd.read_excel("US_Elections.xlsx",sheet_name="Hex")
# us_hex_df.fillna('',axis=1,inplace=True)

# with open('Data/GB/gb_with_results_4simp.geojson') as f:
#     gj = geojson.load(f)
# with open('Data/GB/ni_with_results.geojson') as f:
#     gj_ni = geojson.load(f)
# with open('Data/GB/gb_const_region2005.geojson') as f:
#     gj2005 = geojson.load(f)

In [6]:
# Hex data
uk_hex_df = pd.read_excel("UK_Elections.xlsm",sheet_name="Hex")
uk_hex_df.fillna('',axis=1,inplace=True)

hex_dict = {
            'UK':{'elections':{'2019':'2010 Hex',
                               '2017':'2010 Hex',
                               '2015':'2010 Hex',
                               '2010':'2010 Hex',
                               '2005':'2005 Hex',
                               '2001':'1997 Hex',
                               '1997':'1997 Hex',
                               '1992':'1992 Hex',
                               '1987':'1983 Hex',
                               '1983':'1983 Hex'},
                  'df':uk_hex_df,
                  'Name':'Constituency'},
#              'US':{'elections':{'2020':'HexWDC',},
#                    'df':us_hex_df,
#                    'Name':'State'},
#             'Canada':{'2021':hex_df}
           }

In [11]:
# Main GUI

class GUI():
    
    def __init__(self,country='UK',election='2019',input_dict=map_dict):
        
        self.country = country
        self.election = election
        self.input_dict = input_dict
        self.title = widgets.HTML("<h2>United Kingdom General Elections</h2>")
        self.country_option = widgets.ToggleButtons(options=['UK'], #list(self.input_dict.keys()),
                                                layout=Layout(margin="10px 0px 20px 0px"),
                                                button_style='danger')
        self.election_option = widgets.ToggleButtons(options=list(self.input_dict[self.country]['elections'].keys()),
                                        layout=Layout(margin="10px 0px 20px 0px"),
                                        button_style='warning')
        self.us_option = widgets.ToggleButtons(options=['President','House','Senate'],
                                        layout=Layout(margin="10px 0px 20px 0px"),
                                        button_style='success')
        self.country_option.observe(self.switch_country,names='value')
        self.election_option.observe(self.switch_election,names='value')
        self.view_option = widgets.ToggleButtons(options=['Map','Hex'],button_style='info')
        self.view_option.observe(self.switch_view,names='value')
        
        self.over_label = widgets.HTML("<b>Constituency at cursor:</b>")
        self.info_box_over = widgets.Text()
        self.info_box_click = widgets.Output()
        self.info_box = widgets.VBox([self.over_label,self.info_box_over,self.info_box_click],layout=Layout(width="40%"))
        
        self.map_output = widgets.VBox()
        self.hex_output = widgets.Output()
        self.svg_output = widgets.Output()
        
        self.hex_box = widgets.VBox([self.view_option,self.hex_output],layout=Layout(width='100%'))
        self.map_box = widgets.VBox([self.view_option,self.map_output],layout=Layout(width='65%',height='900px'))
        self.svg_box = widgets.VBox([self.view_option,self.svg_output],layout=Layout(width='100%'))
        
        self.map_info_box = widgets.HBox([self.map_box,self.info_box],layout=Layout(height="900px",width="100%"))
        
        self.menu_options = widgets.ToggleButtons(options=['Elections','Constituencies'], button_style='success')
        self.menu_options.observe(self.switch_detail,names='value')
        self.detail_list = widgets.Dropdown(options=ELECTION.elect_list.keys())
        self.detail_list.observe(self.load_detail,names='value')
        self.menu_bar = widgets.HBox([self.menu_options,self.detail_list])
        self.detail_output = widgets.Output()
        self.detail_box = widgets.VBox([self.menu_bar,self.detail_output],layout=Layout(width="100%"))
        
        self.gui = widgets.VBox([self.title,self.country_option,self.election_option,self.map_info_box,self.detail_box],layout=Layout(align_items='center'))
        
        #self.load_map()
        self.load_detail('')
    
    def change_title(new_title):
        
        self.title.value = "<h2>" + str(new_title) + "<\h2>"
    
    def get_inputs(self):
        
        country = self.country_option.value
        election = self.election_option.value
        view = self.view_option.value
        if view == 'Map':
            input_dict = map_dict
        elif view == 'Hex':
            input_dict = hex_dict
            
        return country, election, view, input_dict
    
    def switch_country(self,wid):
        
        country, election, view, input_dict = self.get_inputs()
        
        self.election_option.options = list(map_dict[country]['elections'].keys())
        if country not in hex_dict:
            self.view_option.options = ['Map']
        else:
            self.view_option.options = ['Map','Hex']
        if country == 'US':
            self.gui.children = [self.title,self.country_option,self.election_option,self.us_option,self.map_info_box]
        else:
            self.gui.children = [self.title,self.country_option,self.election_option,self.map_info_box]
        self.load_view()
    
    def switch_election(self,wid):
        
        country, election, view, input_dict = self.get_inputs()
        
        if view == 'Map':
            if country in hex_dict and election in hex_dict[country]['elections']:
                self.view_option.options = ['Map','Hex']
            else:
                self.view_option.options = ['Map']
        elif view == 'Hex':
            if country in map_dict and election in map_dict[country]['elections']:
                self.view_option.options = ['Map','Hex']
            else:
                self.view_option.options = ['Hex']
        self.load_view()
    
    def switch_view(self,wid):
        
        if self.view_option.value == 'Map':
            self.election_option.options = list(map_dict[self.country_option.value]['elections'].keys())
        if self.view_option.value == 'Hex':
            self.election_option.options = list(hex_dict[self.country_option.value]['elections'].keys())
        self.load_view()
    
    def switch_detail(self,wid):
        
        detail_type = self.menu_options.value
        
        if detail_type == 'Elections':
            self.detail_list.options = ELECTION.elect_list.keys()
        elif detail_type == 'Constituencies':
            self.detail_list.options = sorted(list(CONSTITUENCY.const_list.keys()))
        
    def load_detail(self,wid):
        
        detail_type = self.menu_options.value
        
        self.detail_output.clear_output()
        
        with self.detail_output:
            
            if detail_type == 'Elections':
                
                election = self.detail_list.value
                obj = ELECTION.elect_list[election]
                df = obj.result_df
                
                html = '''<style>
                        table {
                          font-family: arial, sans-serif;
                          border-collapse: collapse;
                          width: 60%;
                        }

                        td {
                          border: 0.5px solid #000000;
                          text-align: left;
                          padding: 1px 5px 1px 5px;
                        }
                        
                        th {
                          border: 0.5px solid #000000;
                          background-color: #E7E6E7;
                          text-align: center;
                          padding: 1px 5px 1px 5px; 
                        }
                        
                        .blank {
                          border: none;
                        }
                        </style>'''   
                
                if obj.end_date == '':
                    html += '<div><b>Date: </b>' + str(int(obj.start_date.strftime('%d'))) + \
                            obj.start_date.strftime(' %b %Y') + '</div>'
                else:
                    html += '<div><b>Date: </b>' + str(int(obj.start_date.strftime('%d'))) + \
                            obj.start_date.strftime(' %b %Y') + ' - ' + str(int(obj.end_date.strftime('%d'))) + \
                            obj.end_date.strftime(' %b %Y') + '</div>'
                html += '<div><b>Largest Party: </b>' + obj.largest_party + '</div>'
                html += '<div><b>Prime Minister: </b>' + obj.pm + '</div>'
                #html += '<div><b>Turnout: </b>' + obj.turnout + '</div>'
                
                seats = 0
                res = ELECTION.elect_list[election].result_df
                for row in res.index:
                    seats += res.Seats[row]
                html += '<div><b>Total Seats: </b>' + str(seats) + '</div>'
                html += '<div><b>Total Votes: </b>' + '{:,}'.format(obj.total_votes) + '</div>'
                
                html += '<table><tr><th colspan="2">Party</th><th>Seats</th><th>Votes</th><th>Percent</th></tr>'
                
                for row in df.index:
                    party = PARTY.party_list.get(df.Party[row],'')
                    if party == '':
                        party_colour = '#C8C7C9'
                    else:
                        party_colour = party.colour
                    html += '<tr><td style="background-color:' + party_colour + ';">  </td>'
                    html += '<td>' + df.Party[row] + '</td>'
                    html += '<td style="text-align:center;">' + str(df.Seats[row]) + '</td>'
                    html += '<td style="text-align:center;">' + '{:,}'.format(df.Votes[row]) + '</td>'
                    html += '<td style="text-align:center;">' + '{:.1f}'.format(df.Percent[row]) + '</td></tr>'
                
            elif detail_type == 'Constituencies':
            
                name = self.detail_list.value
                obj = CONSTITUENCY.const_list[name]
                html = '''<style>
                        table {
                          font-family: arial, sans-serif;
                          border-collapse: collapse;
                          width: 60%;
                        }

                        td {
                          border: 0.5px solid #000000;
                          text-align: left;
                          padding: 1px 5px 1px 5px;
                        }
                        
                        th {
                          border: 0.5px solid #000000;
                          background-color: #E7E6E7;
                          text-align: center;
                          padding: 1px 5px 1px 5px; 
                        }
                        
                        .blank {
                          border: none;
                        }
                        </style>'''                        
                
                html += '<div><b>Region: </b>' + obj.region + '</div>'
                html += '<div><b>County: </b>' + obj.county + '</div>'
                html += '<div><b>Created: </b>' + obj.created.strftime('%Y')
                if obj.predecessors1 == '':
                    html += '</div>'
                else:
                    html += '&nbsp&nbsp&nbsp&nbsp<b>Predecessors: </b>' + obj.predecessors1 + '</div>'
                if obj.abolished != []:
                    for i in range(1,len(obj.abolished)+1):
                        html += '<div><b>Abolished: </b>' + obj.abolished[i-1].strftime('%Y')
                        html += '&nbsp&nbsp&nbsp&nbsp<b>Successors: </b>' + obj.successors.split('|')[i-1] + '</div>'
                        if len(obj.recreated) >= i:
                            html += '<div><b>Recreated: </b>' + obj.recreated[i-1].strftime('%Y')
                            html += '&nbsp&nbsp&nbsp&nbsp<b>Predecessors: </b>' + obj.predecessors2.split('|')[i-1]
                            html += '</div>'
                
                if obj.previous != '':
                    for i in range(0,len(obj.previous.split('|'))):
                        html += '<div><b>Previous Name: </b>' + obj.previous.split('|')[i]
                        html += '<b>Date changed: </b>' + obj.changed[i].strftime('%Y') + '</div>'
                if obj.singleMP != []:
                    html += '<div>Represented by one MP from ' + obj.singleMP[0].strftime('%Y') + ' until '
                    html += obj.singleMP[1].strftime('%Y') + '</div>'
                if obj.doubleMP != []:
                    html += '<div>Represented by two MPs from ' + obj.doubleMP[0].strftime('%Y') + ' until '
                    html += obj.doubleMP[1].strftime('%Y') + '</div>'
                if obj.tripleMP != []:
                    html += '<div>Represented by three MPs from ' + obj.tripleMP[0].strftime('%Y') + ' until '
                    html += obj.tripleMP[1].strftime('%Y') + '</div>'
                
                for election in sorted(list(obj.election_list.keys()), reverse=True):
                    if election[-1] == 'B':
                        if election[-2] == 'B':
                            byelection = election[:-2]
                        else:
                            byelection = election[:-1]
                        html += '<h3 style="text-align:left;">' + byelection + ' By-Election Results</h3>'
                    else:
                        html += '<h3 style="text-align:left;">' + election + ' General Election Results</h3>'
                    
                    res_obj = obj.election_list[election]
                    df = res_obj.results.copy()
                    notes = ''
                    html += '<table><tr><th colspan="2">Party</th><th>Candidate</th><th>Votes</th><th>Percent</th></tr>'
                    for row in df.index:
                        party = PARTY.party_list.get(df.Party[row],'')
                        if party == '':
                            party_colour = '#C8C7C9'
                        else:
                            party_colour = party.colour
                        html += '<tr><td style="background-color:' + party_colour + ';">  </td>'
                        html += '<td>' + df.Party[row] + '</td>'
                        html += '<td>' + df.Candidate[row] + '</td>'
                        if df.Votes[row] == 'Unopposed':
                            html += '<td colspan=2 style="text-align:center;">Unopposed</td></tr>'
                        else:
                            html += '<td style="text-align:center;">' + '{:,}'.format(df.Votes[row]) + '</td>'
                            html += '<td style="text-align:center;">' + '{:.1f}'.format(df.Percent[row]) + '</td></tr>'
                            if df.Notes[row] != '':
                                notes += '<div>' + df.Notes[row] + '</div>'
                    if not res_obj.unopposed:
                        majority_percent = res_obj.majority / res_obj.turnoutN * 100
                        html += '<tr><td colspan="2" class="blank"></td><th>Majority</th>'
                        html += '<td style="text-align:center;">' + '{:,}'.format(res_obj.majority) + '</td>'
                        html += '<td style="text-align:center;">' + '{:.1f}'.format(majority_percent) + '</td></tr>'
                        html += '<tr><td colspan="2" class="blank"></td><th>Turnout</th>'
                        html += '<td style="text-align:center;">' + '{:,}'.format(res_obj.turnoutN) + '</td>'
                        if isinstance(res_obj.turnoutP,str):
                            html += '<td style="text-align:center;">' + res_obj.turnoutP + '</td></tr>'
                        else:
                            html += '<td style="text-align:center;">' + '{:.1f}'.format(res_obj.turnoutP) + '</td></tr>'
                        html += '</table>'
                    else:
                        html += '</table>'
                    if res_obj.notes != '':
                        notes += '<div>' + res_obj.notes + '</div>'
                    html += notes
        
            display(widgets.HTML(html))
        
    def load_view(self):
        
        country, election, view, input_dict = self.get_inputs()      
        
        if self.view_option.value == 'Map':
            if isinstance(input_dict[country]['elections'][election],str):
                self.map_info_box.children = [self.svg_box]
                self.load_svg()
            else:
                self.map_info_box.children = [self.map_box,self.info_box]
                self.load_map()

        elif self.view_option.value == 'Hex':
            self.map_info_box.children = [self.hex_box]
            self.load_hex()

    def load_hex(self):
        
        def get_hex_coords(hex_coords):
    
            final_coords = []
            for hex_string in hex_coords:
                hex_split = hex_string.split(',')
                hex_split = [int(x) for x in hex_split]
                final_coords.append(hex_split)

            # Horizontal cartesian coords
            hcoord_n = [c[0] for c in final_coords]
            # Vertical cartersian coords
            vcoord_n = [2. * np.sin(np.radians(60)) * (c[1] - c[2]) /3. for c in final_coords]

            x_list = []
            y_list = []
            for i in range(0,len(hcoord_n)):
                hex1 = RegularPolygon((hcoord_n[i], vcoord_n[i]), numVertices=6, radius=2. / 3.,orientation=np.radians(30))
                points = hex1.get_verts().tolist()
                x_list.append([m[0] for m in points])
                y_list.append([m[1] for m in points])        

            return x_list, y_list
        
        self.hex_output.clear_output()
        
        country, election, view, input_dict = self.get_inputs()
        
        df = input_dict[country]['df']
        df = df[df[input_dict[country]['elections'][election]] != ""]
        df = df[df[input_dict[country]['elections'][election]] != "Yes"]
        
        with self.hex_output:
            
            names = list(df[input_dict[country]['Name']])
            colours = self.get_colours(names,election,'party')
            coords = list(df[input_dict[country]['elections'][election]])
            xs, ys = get_hex_coords(coords)
            results = [CONSTITUENCY.const_list[x].election_list[election].results.to_dict() for x in names]
            
            count = 0
            for const in results:
                for key in const.keys():
                    for subkey in const[key].keys():
                        results[count][key][subkey] = str(results[count][key][subkey])
                count += 1
            
            data=dict(
                x=xs,
                y=ys,
                name=names,
                colours=colours,
                results = results
            )

            cds = ColumnDataSource(data)

            TOOLS = "pan,wheel_zoom,reset,hover,save"

            p = figure(
                title="General Election " + election, tools=TOOLS,
                x_axis_location=None, y_axis_location=None,
                tooltips=[("Name", "@name")],)

            p.grid.grid_line_color = None
            p.hover.point_policy = "follow_mouse"

            patch_renderer  = p.patches('x', 'y', source=cds,
                      fill_color={"field":"colours"},
                      fill_alpha=0.7, line_color="white", line_width=0.5)

            div = Div(width=400)
            indicator_div = Div(text="",width=350)
            layout = bkCol(bkRow(p, indicator_div))

            tap_tool = TapTool(renderers=[patch_renderer])
            p.add_tools(tap_tool)
            #patch_indicator_callback = CustomJS(args=dict(cds=cds, div=indicator_div), code="""""")
            patch_indicator_callback = CustomJS(args=dict(cds=cds, div=indicator_div, election=election), 
                                                code=self.get_bokeh_code())

            cds.selected.js_on_change('indices', patch_indicator_callback)

            show(layout)

    def load_map(self):
        
        def get_party_colour(feature):
            
            election = self.election_option.value
            const = feature['properties']['name']
            try:
                const_obj = CONSTITUENCY.const_list[const]
            except:
                const_obj = CONSTITUENCY.previous_list[const]
            party = const_obj.election_list[election].party1
            if party in PARTY.party_list.keys():
                colour = PARTY.party_list[party].colour
            else:
                colour = '#FFFFFF'
            return {'fillColor':colour,
                    'color':'#000000',
                    'weight':1,
                    'fillOpacity':0.8}
        
        self.map_output.children = [widgets.HTML("<br><br><br>Loading...")]
        
        country = self.country_option.value
        election = self.election_option.value
        gj = map_dict[country]['elections'][election]
        
        geo_json = GeoJSON(
            data=gj,
            style={'opacity': 1, 'fillOpacity': 0.2, 'weight': 1},
            hover_style={'color': 'black', 'dashArray': '0', 'fillOpacity': 0.5},
            style_callback=get_party_colour
        )

        geo_json.on_hover(self.hover_func)
        geo_json.on_click(self.click_func)    
        
        m = Map(center=(54, 0), zoom=5, scroll_wheel_zoom=True, layout=Layout(height="700px"))
        m.add_layer(geo_json)
        
        self.map_output.children = [m]
        
    def load_svg(self):
        
        def parse_svg(root,election):
    
            def transform_path(path,transform,election=''):

                if transform[:6] == 'matrix':
                    a,b,c,d,e,f = [float(i) for i in transform[7:-1].split(',')]
                    count = 0
                    for coords in path.vertices:
                        x = a * coords[0] + c * coords[1] + e
                        y = b * coords[0] + d * coords[1] + f
                        path.vertices[count][0] = x
                        path.vertices[count][1] = y
                        count += 1

                elif transform[:9] == 'translate':
                    coords_split = [float(i) for i in transform[10:-1].split(',')]
                    if len(coords_split) == 2:
                        x_shift, y_shift = coords_split
                    count = 0
                    for coords in path.vertices:
                        x = coords[0] + x_shift
                        y = coords[1] + y_shift
                        path.vertices[count][0] = x
                        path.vertices[count][1] = y
                        count += 1

                elif transform == 'Shetland':
                    
                    if election in ['1992','1987','1983']:
                        a,b,c,d,e,f = [0.3,0,0,0.3,60,-190]
                    elif election in ['2001','1997']:
                        a,b,c,d,e,f = [0.7,0,0,0.7,-60,-180]

                    count = 0
                    for coords in path.vertices:
                        x = a * coords[0] + c * coords[1] + e
                        y = b * coords[0] + d * coords[1] + f
                        path.vertices[count][0] = x
                        path.vertices[count][1] = y
                        count += 1 

                return path

            def check_element(paths, element, transform, election):

                if 'transform' in element.keys():
                    transform = element.attrib['transform']

                if element.tag != '{http://www.w3.org/2000/svg}path':
                    for e in element:
                        paths = check_element(paths,e,transform,election)
                else:
                    if 'd' in element.keys() and element.attrib['d'] != '':
                        
                        path = element.attrib['d']
                        parsed = parse_path(path)
                        if element.attrib['id'] == 'Orkney and Shetland':
                            parsed = transform_path(parsed,'Shetland',election=election)
                        elif transform:
                            parsed = transform_path(parsed,transform)
                        if parsed:
                            paths.append(parsed)
                        
                        return paths

                return paths

            tree = etree.parse(filename)
            root = tree.getroot()
            paths = []

            for element in root:
                transform = None
                paths = check_element(paths,element,transform,election)

            return paths

        def convert_paths(paths, names):
            
            def convert_path_into_polygons(path,pg_x,pg_y,count=-1):

                if count == -1:

                    vertices = path.vertices
                    codes = path.codes

                    cur_pg = []

                    for i in range(0,len(vertices)):
                        if codes[i] == 79:
                            pg_x.append([[c[0] for c in cur_pg]])
                            pg_y.append([[c[1]*-1 for c in cur_pg]]) # multiply by -1 to invert
                            cur_pg = []
                        else:
                            cur_pg.append(vertices[i])

                elif names[count] == 'Milton Keynes North East':

                    vertices = []
                    vertices += list(path.vertices[:6])
                    vertices += list(paths[count+1].vertices[1:10])
                    vertices += list(path.vertices[15:])
                    codes = [1,]
                    codes += [2] * (len(vertices)-2) 
                    codes.append(79)

                    cur_pg = []
                    names[count] = 'Milton Keynes'

                    for i in range(0,len(vertices)):
                        if codes[i] == 79:
                            pg_x.append([[c[0] for c in cur_pg]])
                            pg_y.append([[c[1]*-1 for c in cur_pg]]) # multiply by -1 to invert
                            cur_pg = []
                        else:
                            cur_pg.append(vertices[i])

                return pg_x, pg_y
            
            xs = []
            ys = []
            x_temp = []
            y_temp = []
            anomalies = {}
            count = 0
            
            for path in paths:
                
                if 'Milton' in names[count] and election in ('1987','1983'):
                    x_temp, y_temp = convert_path_into_polygons(path,x_temp,y_temp,count)
                    if names[count] == 'Milton Keynes':
                        xs.append(x_temp)
                        ys.append(y_temp)
                    x_temp = []
                    y_temp = []
                elif election in ('1992','1987','1983') and names[count] in ('path4893','Glasgow Central'):
                    anomalies[names[count]] = path
                else:    
                    x_temp, y_temp = convert_path_into_polygons(path,x_temp,y_temp)
                    if count == len(paths)-1 or names[count+1] != names[count]:       
                        xs.append(x_temp)
                        ys.append(y_temp)
                        x_temp = []
                        y_temp = []

                count += 1
            
            return xs, ys, anomalies
        
        def correct_glasgow(names,xs,ys,anomalies):
    
            names.remove('Glasgow Central')
            names.remove('path4893')
            names.append('Glasgow Central')

            vertices = []
            vertices += list(anomalies['path4893'].vertices[0:11])
            vertices += list(anomalies['Glasgow Central'].vertices[0:6])
            vertices += list(anomalies['path4893'].vertices[16:])
            codes = [1,]
            codes += [2] * (len(vertices)-2) 
            codes.append(79)

            cur_pg = []

            for i in range(0,len(vertices)):
                if codes[i] == 79:
                    xs.append([[[c[0] for c in cur_pg]]])
                    ys.append([[[c[1]*-1 for c in cur_pg]]]) # multiply by -1 to invert
                    cur_pg = []
                else:
                    cur_pg.append(vertices[i])

            return names, xs, ys

        def check_for_holes(xs, ys):
    
            const_holes = {('1983','1987','1992'):
                               {'PENRITH AND THE BORDER':'CARLISLE',
                                'WANSDYKE':'BATH',
                                'CIRENCESTER AND TEWKESBURY':'CHELTENHAM'},
                       ('1997','2001'):
                               {'NORTH ESSEX':'COLCHESTER'}
                      }

            for key in const_holes.keys():

                if election in key:

                    for const in const_holes[key].keys():

                        count = 0
                        outer = 0
                        inner = 0
                        for name in names:
                            if name.upper() == const:
                                outer = count
                            elif name.upper() == const_holes[key][const]:
                                inner = count
                            count += 1

                        xs[outer][0].append(xs[inner][0][0])
                        ys[outer][0].append(ys[inner][0][0])

            return xs, ys
        
        self.svg_output.clear_output()
        
        with self.svg_output:
            
            country, election, view, input_dict = self.get_inputs()
            filename = "SVGs/" + country + input_dict[country]['elections'][election] + ".svg"

            tree = etree.parse(filename)
            root = tree.getroot()
            path_elems = root.findall('.//{http://www.w3.org/2000/svg}path')
            names = [elem.attrib['id'] for elem in path_elems]
            paths = parse_svg(root,election)

            xs, ys, anomalies = convert_paths(paths, names)
            
            if election in ('1987','1983'):
                names.remove('Milton Keynes South West')
            if election in ('1992','1987','1983'):
                names, xs, ys = correct_glasgow(names,xs,ys,anomalies)

            xs, ys = check_for_holes(xs,ys)

            for i in range(len(names)-1,-1,-1):
                if i != 0 and names[i]==names[i-1]:
                    names.pop(i)

            colours = self.get_colours(names, election)
            
            results = []
            for name in names:
                if name in CONSTITUENCY.const_list.keys():
                    results.append(CONSTITUENCY.const_list[name].election_list[election].results.to_dict())
                else:
                    results.append(CONSTITUENCY.previous_list[name].election_list[election].results.to_dict())
            count = 0
            for const in results:
                for key in const.keys():
                    for subkey in const[key].keys():
                        results[count][key][subkey] = str(results[count][key][subkey])
                count += 1
            
            data=dict(
                x=xs,
                y=ys,
                name=names,
                colours=colours,
                results = results
            )

            cds = ColumnDataSource(data)

            TOOLS = "pan,wheel_zoom,reset,hover,save"

            p = figure(
                title="General Election " + election, tools=TOOLS,
                x_axis_location=None, y_axis_location=None,
                tooltips=[("Name", "@name")],
                aspect_ratio=0.7)

            patch_renderer = p.multi_polygons(xs="x",ys="y",line_width=1,color="colours",line_color='black',
                                              name="names",source=cds)
            
            p.hover.point_policy = "follow_mouse"

            div = Div(width=400)
            indicator_div = Div(text="",width=350)
            layout = bkCol(bkRow(p, indicator_div))
            
            tap_tool = TapTool(renderers=[patch_renderer])
            p.add_tools(tap_tool)
            display_text = self.get_bokeh_code()
            patch_indicator_callback = CustomJS(args=dict(cds=cds, div=indicator_div, election=election), 
                                                code=display_text)
            
            cds.selected.js_on_change('indices', patch_indicator_callback)
            
            show(layout)

    def get_colours(self, consts, election, mode='party'):

        colours = []
        default_colour_scale = ['#403943','#5F5763','#7F7484','#A199A5','#C4BEC6']
        
        if mode[:2] == 'SP':
            
            party = mode[2:]
            coalition = ''
            if PARTY.party_list[party].coalition != '':
                coalition = PARTY.party_list[party].coalition
            
            if party in PARTY.party_list and PARTY.party_list[party].colour_scale != []:
                colour_scale = PARTY.party_list[party].colour_scale
            else:
                colour_scale = ['#403943','#5F5763','#7F7484','#A199A5','#C4BEC6']
              
            percents = []
            
            for const in consts:
                if const not in CONSTITUENCY.const_list.keys():
                    try:
                        const_obj = CONSTITUENCY.previous_list[const]
                    except:
                        raise Exception('Constituency not in database',const,election)
                else:
                    const_obj = CONSTITUENCY.const_list[const]
                df = const_obj.election_list[election].results
                if party in list(df.Party):
                    percent_series = df.loc[df['Party'] == party, 'Percent']
                    if percent_series[percent_series.index[0]] != '':
                        percents.append(percent_series[percent_series.index[0]])
            
            arrays = np.array_split(sorted(percents,reverse=True),5)
            
            const_count = 0
            colour_count = 0
            for const in consts:
                if const not in CONSTITUENCY.const_list.keys():
                    try:
                        const_obj = CONSTITUENCY.previous_list[const]
                    except:
                        raise Exception('Constituency not in database',const,election)
                else:
                    const_obj = CONSTITUENCY.const_list[const]
                df = const_obj.election_list[election].results
                if party in list(df.Party):
                    percent_series = df.loc[df['Party'] == party, 'Percent']
                    percent = percent_series[percent_series.index[0]]
                    count = 0
                    for array in arrays:
                        if percent in array:
                            colour_count += 1
                            colours.append(colour_scale[count])
                            break
                        count += 1
                else:
                    colours.append('#C3C4BE')
                const_count += 1
                #print(const_count,colour_count,end=':')
                #if colour_count > 650:
                    #print(const)
            
            return colours
        
        for const in consts:

            if const not in CONSTITUENCY.const_list.keys():
                try:
                    const_obj = CONSTITUENCY.previous_list[const]
                except:
                    raise Exception('Constituency not in database',const,election)
            else:
                const_obj = CONSTITUENCY.const_list[const]

            if mode == 'party':

                try:
                    results = const_obj.election_list[election]
                except:
                    raise Exception('Election results not found',const,election)

                winner = results.party1

                try:
                    colours.append(PARTY.party_list[winner].colour)
                except:
                    raise Exception('Winning party not found',winner)
                
            elif mode == 'county':

                county = const_obj.county
                colours.append(county_colours.get(county,'pink'))

        return colours

    def get_bokeh_code(self):
        
        country, election, view, input_dict = self.get_inputs()
        display_text = ""
        
        if country == 'UK':
            display_text = """div.text = "<h2>" + cds.data['name'][cb_obj.indices] + "</h2><h3>" + election + " General Election Results</h3><br>" + 
                      "<b>Winning Party:</b> " + cds.data['results'][cb_obj.indices]['Party'][0] + "<br><br>" +
                      "<table>" +
                      "<tr>" +
                        "<th>Party</th>" +
                        "<th>Candidate</th>" +
                        "<th>Votes</th>" +
                        "<th>Percent</th>" +
                      "</tr>" +
                      "<tr>"
                      let partyList = cds.data['results'][cb_obj.indices]['Party']
                      for (let i = 0; i < Object.keys(partyList).length; i++) {
                        div.text += "<td>" + cds.data['results'][cb_obj.indices]['Party'][i] + "</td>";
                        div.text += "<td>" + cds.data['results'][cb_obj.indices]['Candidate'][i] + "</td>";
                        div.text += "<td>" + cds.data['results'][cb_obj.indices]['Votes'][i] + "</td>";
                        div.text += "<td>" + cds.data['results'][cb_obj.indices]['Percent'][i] + "</td></tr>";
                        }
                      div.text += "</table>"
                   """
            
        return display_text
    
    def hover_func(self,event,feature,properties):
        
        self.info_box_over.value = str(properties['name'])

    def click_func(self,event,feature,properties):

        self.info_box_click.clear_output()

        with self.info_box_click:
            
            const = properties['name']
            const_obj = CONSTITUENCY.const_list[const]
            results = const_obj.election_list[self.election_option.value]
            title = widgets.HTML("<h3>" + const + "</h3><br><h4>" + self.election_option.value + " Results</h4>")
            display(title)
            df = results.results
            df = df[['Party','Candidate','Votes','Percent']]
            df.set_index('Party',inplace=True)
            df = df.rename({'Percent':'%'},axis=1)
            df['Votes'] = df['Votes'].apply(lambda x: "{:,}".format(x))
            df['%'] = df['%'].apply(lambda x: "{:.1f}".format(x))
            col_defs = {'Votes':{'width':60},'%':{'width':40}}
            display(qgrid.show_grid(df,grid_options={'maxVisibleRows':len(df),'minVisibleRows':len(df)},column_definitions=col_defs))

gui = GUI()

display(gui.gui)



In [57]:
# US results
results='''Alabama 1	Republican
Alabama 2	Republican
Alabama 3	Republican
Alabama 4	Republican
Alabama 5	Republican
Alabama 6	Republican
Alabama 7	Democratic
Alaska 0	Republican
Arizona 1	Democratic
Arizona 2	Democratic
Arizona 3	Democratic
Arizona 4	Republican
Arizona 5	Republican
Arizona 6	Republican
Arizona 7	Democratic
Arizona 8	Republican
Arizona 9	Democratic
Arkansas 1	Republican
Arkansas 2	Republican
Arkansas 3	Republican
Arkansas 4	Republican
California 1	Republican
California 2	Democratic
California 3	Democratic
California 4	Republican
California 5	Democratic
California 6	Democratic
California 7	Democratic
California 8	Republican
California 9	Democratic
California 10	Democratic
California 11	Democratic
California 12	Democratic
California 13	Democratic
California 14	Democratic
California 15	Democratic
California 16	Democratic
California 17	Democratic
California 18	Democratic
California 19	Democratic
California 20	Democratic
California 21	Republican
California 22	Republican
California 23	Republican
California 24	Democratic
California 25	Republican
California 26	Democratic
California 27	Democratic
California 28	Democratic
California 29	Democratic
California 30	Democratic
California 31	Democratic
California 32	Democratic
California 33	Democratic
California 34	Democratic
California 35	Democratic
California 36	Democratic
California 37	Democratic
California 38	Democratic
California 39	Republican
California 40	Democratic
California 41	Democratic
California 42	Republican
California 43	Democratic
California 44	Democratic
California 45	Democratic
California 46	Democratic
California 47	Democratic
California 48	Republican
California 49	Democratic
California 50	Republican
California 51	Democratic
California 52	Democratic
California 53	Democratic
Colorado 1	Democratic
Colorado 2	Democratic
Colorado 3	Republican
Colorado 4	Republican
Colorado 5	Republican
Colorado 6	Democratic
Colorado 7	Democratic
Connecticut 1	Democratic
Connecticut 2	Democratic
Connecticut 3	Democratic
Connecticut 4	Democratic
Connecticut 5	Democratic
Delaware 0	Democratic
Florida 1	Republican
Florida 2	Republican
Florida 3	Republican
Florida 4	Republican
Florida 5	Democratic
Florida 6	Republican
Florida 7	Democratic
Florida 8	Republican
Florida 9	Democratic
Florida 10	Democratic
Florida 11	Republican
Florida 12	Republican
Florida 13	Democratic
Florida 14	Democratic
Florida 15	Republican
Florida 16	Republican
Florida 17	Republican
Florida 18	Republican
Florida 19	Republican
Florida 20	Democratic
Florida 21	Democratic
Florida 22	Democratic
Florida 23	Democratic
Florida 24	Democratic
Florida 25	Republican
Florida 26	Republican
Florida 27	Republican
Georgia 1	Republican
Georgia 2	Democratic
Georgia 3	Republican
Georgia 4	Democratic
Georgia 5	Democratic
Georgia 6	Democratic
Georgia 7	Democratic
Georgia 8	Republican
Georgia 9	Republican
Georgia 10	Republican
Georgia 11	Republican
Georgia 12	Republican
Georgia 13	Democratic
Georgia 14	Republican
Hawaii 1	Democratic
Hawaii 2	Democratic
Idaho 1	Republican
Idaho 2	Republican
Illinois 1	Democratic
Illinois 2	Democratic
Illinois 3	Democratic
Illinois 4	Democratic
Illinois 5	Democratic
Illinois 6	Democratic
Illinois 7	Democratic
Illinois 8	Democratic
Illinois 9	Democratic
Illinois 10	Democratic
Illinois 11	Democratic
Illinois 12	Republican
Illinois 13	Republican
Illinois 14	Democratic
Illinois 15	Republican
Illinois 16	Republican
Illinois 17	Democratic
Illinois 18	Republican
Indiana 1	Democratic
Indiana 2	Republican
Indiana 3	Republican
Indiana 4	Republican
Indiana 5	Republican
Indiana 6	Republican
Indiana 7	Democratic
Indiana 8	Republican
Indiana 9	Republican
Iowa 1	Republican
Iowa 2	Republican
Iowa 3	Democratic
Iowa 4	Republican
Kansas 1	Republican
Kansas 2	Republican
Kansas 3	Democratic
Kansas 4	Republican
Kentucky 1	Republican
Kentucky 2	Republican
Kentucky 3	Democratic
Kentucky 4	Republican
Kentucky 5	Republican
Kentucky 6	Republican
Louisiana 1	Republican
Louisiana 2	Democratic
Louisiana 3	Republican
Louisiana 4	Republican
Louisiana 5	Republican
Louisiana 6	Republican
Maine 1	Democratic
Maine 2	Democratic
Maryland 1	Republican
Maryland 2	Democratic
Maryland 3	Democratic
Maryland 4	Democratic
Maryland 5	Democratic
Maryland 6	Democratic
Maryland 7	Democratic
Maryland 8	Democratic
Massachusetts 1	Democratic
Massachusetts 2	Democratic
Massachusetts 3	Democratic
Massachusetts 4	Democratic
Massachusetts 5	Democratic
Massachusetts 6	Democratic
Massachusetts 7	Democratic
Massachusetts 8	Democratic
Massachusetts 9	Democratic
Michigan 1	Republican
Michigan 2	Republican
Michigan 3	Republican
Michigan 4	Republican
Michigan 5	Democratic
Michigan 6	Republican
Michigan 7	Republican
Michigan 8	Democratic
Michigan 9	Democratic
Michigan 10	Republican
Michigan 11	Democratic
Michigan 12	Democratic
Michigan 13	Democratic
Michigan 14	Democratic
Minnesota 1	Republican
Minnesota 2	Democratic
Minnesota 3	Democratic
Minnesota 4	Democratic
Minnesota 5	Democratic
Minnesota 6	Republican
Minnesota 7	Republican
Minnesota 8	Republican
Mississippi 1	Republican
Mississippi 2	Democratic
Mississippi 3	Republican
Mississippi 4	Republican
Missouri 1	Democratic
Missouri 2	Republican
Missouri 3	Republican
Missouri 4	Republican
Missouri 5	Democratic
Missouri 6	Republican
Missouri 7	Republican
Missouri 8	Republican
Montana 0	Republican
Nebraska 1	Republican
Nebraska 2	Republican
Nebraska 3	Republican
Nevada 1	Democratic
Nevada 2	Republican
Nevada 3	Democratic
Nevada 4	Democratic
New Hampshire 1	Democratic
New Hampshire 2	Democratic
New Jersey 1	Democratic
New Jersey 2	Republican
New Jersey 3	Democratic
New Jersey 4	Republican
New Jersey 5	Democratic
New Jersey 6	Democratic
New Jersey 7	Democratic
New Jersey 8	Democratic
New Jersey 9	Democratic
New Jersey 10	Democratic
New Jersey 11	Democratic
New Jersey 12	Democratic
New Mexico 1	Democratic
New Mexico 2	Republican
New Mexico 3	Democratic
New York 1	Republican
New York 2	Republican
New York 3	Democratic
New York 4	Democratic
New York 5	Democratic
New York 6	Democratic
New York 7	Democratic
New York 8	Democratic
New York 9	Democratic
New York 10	Democratic
New York 11	Republican
New York 12	Democratic
New York 13	Democratic
New York 14	Democratic
New York 15	Democratic
New York 16	Democratic
New York 17	Democratic
New York 18	Democratic
New York 19	Democratic
New York 20	Democratic
New York 21	Republican
New York 22	Republican
New York 23	Republican
New York 24	Republican
New York 25	Democratic
New York 26	Democratic
New York 27	Republican
North Carolina 1	Democratic
North Carolina 2	Democratic
North Carolina 3	Republican
North Carolina 4	Democratic
North Carolina 5	Republican
North Carolina 6	Democratic
North Carolina 7	Republican
North Carolina 8	Republican
North Carolina 9	Republican
North Carolina 10	Republican
North Carolina 11	Republican
North Carolina 12	Democratic
North Carolina 13	Republican
North Dakota 0	Republican
Ohio 1	Republican
Ohio 2	Republican
Ohio 3	Democratic
Ohio 4	Republican
Ohio 5	Republican
Ohio 6	Republican
Ohio 7	Republican
Ohio 8	Republican
Ohio 9	Democratic
Ohio 10	Republican
Ohio 11	Democratic
Ohio 12	Republican
Ohio 13	Democratic
Ohio 14	Republican
Ohio 15	Republican
Ohio 16	Republican
Oklahoma 1	Republican
Oklahoma 2	Republican
Oklahoma 3	Republican
Oklahoma 4	Republican
Oklahoma 5	Republican
Oregon 1	Democratic
Oregon 2	Republican
Oregon 3	Democratic
Oregon 4	Democratic
Oregon 5	Democratic
Pennsylvania 1	Republican
Pennsylvania 2	Democratic
Pennsylvania 3	Democratic
Pennsylvania 4	Democratic
Pennsylvania 5	Democratic
Pennsylvania 6	Democratic
Pennsylvania 7	Democratic
Pennsylvania 8	Democratic
Pennsylvania 9	Republican
Pennsylvania 10	Republican
Pennsylvania 11	Republican
Pennsylvania 12	Republican
Pennsylvania 13	Republican
Pennsylvania 14	Republican
Pennsylvania 15	Republican
Pennsylvania 16	Republican
Pennsylvania 17	Democratic
Pennsylvania 18	Democratic
Rhode Island 1	Democratic
Rhode Island 2	Democratic
South Carolina 1	Republican
South Carolina 2	Republican
South Carolina 3	Republican
South Carolina 4	Republican
South Carolina 5	Republican
South Carolina 6	Democratic
South Carolina 7	Republican
South Dakota 0	Republican
Tennessee 1	Republican
Tennessee 2	Republican
Tennessee 3	Republican
Tennessee 4	Republican
Tennessee 5	Democratic
Tennessee 6	Republican
Tennessee 7	Republican
Tennessee 8	Republican
Tennessee 9	Democratic
Texas 1	Republican
Texas 2	Republican
Texas 3	Republican
Texas 4	Republican
Texas 5	Republican
Texas 6	Republican
Texas 7	Democratic
Texas 8	Republican
Texas 9	Democratic
Texas 10	Republican
Texas 11	Republican
Texas 12	Republican
Texas 13	Republican
Texas 14	Republican
Texas 15	Democratic
Texas 16	Democratic
Texas 17	Republican
Texas 18	Democratic
Texas 19	Republican
Texas 20	Democratic
Texas 21	Republican
Texas 22	Republican
Texas 23	Republican
Texas 24	Republican
Texas 25	Republican
Texas 26	Republican
Texas 27	Republican
Texas 28	Democratic
Texas 29	Democratic
Texas 30	Democratic
Texas 31	Republican
Texas 32	Democratic
Texas 33	Democratic
Texas 34	Democratic
Texas 35	Democratic
Texas 36	Republican
Utah 1	Republican
Utah 2	Republican
Utah 3	Republican
Utah 4	Republican
Vermont 0	Democratic
Virginia 1	Republican
Virginia 2	Democratic
Virginia 3	Democratic
Virginia 4	Democratic
Virginia 5	Republican
Virginia 6	Republican
Virginia 7	Democratic
Virginia 8	Democratic
Virginia 9	Republican
Virginia 10	Democratic
Virginia 11	Democratic
Washington 1	Democratic
Washington 2	Democratic
Washington 3	Republican
Washington 4	Republican
Washington 5	Republican
Washington 6	Democratic
Washington 7	Democratic
Washington 8	Democratic
Washington 9	Democratic
Washington 10	Democratic
West Virginia 1	Republican
West Virginia 2	Republican
West Virginia 3	Republican
Wisconsin 1	Republican
Wisconsin 2	Democratic
Wisconsin 3	Democratic
Wisconsin 4	Democratic
Wisconsin 5	Republican
Wisconsin 6	Republican
Wisconsin 7	Republican
Wisconsin 8	Republican
Wyoming 0	Republican'''

In [18]:
# results = results.split('\n')
# full_results = {}
# for x in results:
#     y = x.split('\t')
#     full_results[y[0]] = y[1]
# full_results

In [19]:
# state_dict={1:'Alabama',2:'Alaska',4:'Arizona',5:'Arkansas',6:'California',8:'Colorado',9:'Connecticut',10:'Delaware',
# 11:'DC',12:'Florida',13:'Georgia',15:'Hawaii',16:'Idaho',17:'Illinois',18:'Indiana',19:'Iowa',20:'Kansas',
# 21:'Kentucky',22:'Louisiana',23:'Maine',24:'Maryland',25:'Massachusetts',26:'Michigan',27:'Minnesota', 
# 28:'Mississippi',29:'Missouri',30:'Montana',31:'Nebraska',32:'Nevada',33:'New Hampshire',34:'New Jersey', 
# 35:'New Mexico',36:'New York',37:'North Carolina',38:'North Dakota',39:'Ohio',40:'Oklahoma',41:'Oregon', 
# 42:'Pennsylvania',44:'Rhode Island',45:'South Carolina',46:'South Dakota',47:'Tennessee',48:'Texas', 
# 49:'Utah',50:'Vermont',51:'Virginia',53:'Washington',54:'West Virginia',55:'Wisconsin',56:'Wyoming',60:'American Samoa',
# 66:'Guam',69:'North Mariana Islands',72:'Puerto Rico',78:'US Virgin Islands'}

# count = 0
# for feature in gj.features:
#     state_num = feature['properties']['STATEFP']
#     state_name = state_dict.get(int(state_num),'Unknown')
#     if state_name == 'Unknown':
#         print(state_num)
#     gj[count]['properties']['STATE_NAME'] = state_name
#     cd_num = feature['properties']['CD116FP']
#     full_cd = state_name + ' ' + str(int(cd_num))
#     if state_name == 'DC' or state_name == 'Unknown':
#         gj[count]['properties']['WINNER'] = 'NONE'
#     else:
#         gj[count]['properties']['WINNER'] = full_results.get(full_cd,'')
#     count += 1

In [ ]:
# def hover_funcx(event,feature,properties):
#     info_box_overx.value = str(properties) # + " " + str(properties['CD'])

# def get_party_colour(feature):
    
#     if feature['properties']['WINNER'] == "Democratic":
#         colour = '#1035de'
#     elif feature['properties']['WINNER'] == "Republican":
#         colour = '#ed220c'
#         #colour = 'YELLOWGREEN'
#     else:
#         colour = '#ffffff'
    
#     return {'fillColor':colour,
#             'color':'#000000',
#             'weight':1,
#             'fillOpacity':0.4}

# m = Map(center=(30, -100), zoom=5, layout=Layout(height="1000px",width="1000px"),scroll_wheel_zoom=True)
# geo_json_can = GeoJSON(
#     data=can_gj,
#     style={'opacity': 1, 'fillOpacity': 0.6, 'weight': 1},
#     hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 0.4},
#     #style_callback=get_party_colour
# )
# geo_json_us = GeoJSON(
#     data=us_gj,
#     style={'opacity': 1, 'fillOpacity': 0.6, 'weight': 1},
#     hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 0.4},
#     #style_callback=get_party_colour
# )
# m.add_layer(geo_json_can)
# m.add_layer(geo_json_us)
# geo_json.on_hover(hover_funcx)

# info_box_overx = widgets.Textarea()
# display(info_box_overx)
# display(m)



In [14]:
###################################################### Break Cell ######################################################

In [ ]:
# Old SVG

# def parse_svg(filename):
    
#     def transform_path(path,transform):
        
#         if transform[:6] == 'matrix':
#             a,b,c,d,e,f = [float(i) for i in transform[7:-1].split(',')]
#             count = 0
#             for coords in path.vertices:
#                 x = a * coords[0] + c * coords[1] + e
#                 y = b * coords[0] + d * coords[1] + f
#                 path.vertices[count][0] = x
#                 path.vertices[count][1] = y
#                 count += 1
        
#         elif transform[:9] == 'translate':
#             x_shift, y_shift = [float(i) for i in transform[10:-1].split(',')]
#             count = 0
#             for coords in path.vertices:
#                 x = coords[0] + x_shift
#                 y = coords[1] + y_shift
#                 path.vertices[count][0] = x
#                 path.vertices[count][1] = y
#                 count += 1
        
#         elif transform == 'Shetland':
#             a= 0.3
#             b = 0
#             c = 0
#             d = 0.3
#             e = 60
#             f = -190
#             count = 0
#             for coords in path.vertices:
#                 x = a * coords[0] + c * coords[1] + e
#                 y = b * coords[0] + d * coords[1] + f
#                 path.vertices[count][0] = x
#                 path.vertices[count][1] = y
#                 count += 1      
                
#         elif transform == 'Shetland97':
#             a= 0.7
#             b = 0
#             c = 0
#             d = 0.7
#             e = -60
#             f = -180
#             count = 0
#             for coords in path.vertices:
#                 x = a * coords[0] + c * coords[1] + e
#                 y = b * coords[0] + d * coords[1] + f
#                 path.vertices[count][0] = x
#                 path.vertices[count][1] = y
#                 count += 1   
            
#         return path
    
#     def check_element(paths, element, transform):
        
#         if 'transform' in element.keys():
#             transform = element.attrib['transform']
        
#         if element.tag != '{http://www.w3.org/2000/svg}path':
#             try:
#                 for e in element:
#                     paths = check_element(paths,e,transform)
#             except Exception as e:
#                 print(e,element.tag)
#         else:
#             if 'd' in element.keys() and element.attrib['d'] != '':
#                 #print('creating path')
#                 path = element.attrib['d']
#                 parsed = parse_path(path)
#                 if element.attrib['id'] == 'Orkney_and_Shetland':
#                     parsed = transform_path(parsed,'Shetland')
#                 elif element.attrib['id'] == 'Orkney and Shetland':
#                     parsed = transform_path(parsed,'Shetland97')
#                 elif transform:
#                     parsed = transform_path(parsed,transform)
#                 if parsed:
#                     paths.append(parsed)
#                 return paths
            
#         return paths
    
#     tree = etree.parse(filename)
#     root = tree.getroot()
#     width = int(re.match(r'\d+', root.attrib['width']).group())
#     height = int(re.match(r'\d+', root.attrib['height']).group())
#     paths = []
    
#     for element in root:
#         transform = None
#         paths = check_element(paths,element,transform)
       
#     return paths, width, height

# def convert_path_into_polygons(path,pg_x,pg_y,count=-1):
    
#     if count == -1:
        
#         vertices = path.vertices
#         codes = path.codes

#         cur_pg = []

#         for i in range(0,len(vertices)):
#             if codes[i] == 79:
#                 pg_x.append([[c[0] for c in cur_pg]])
#                 pg_y.append([[c[1]*-1 for c in cur_pg]]) # multiply by -1 to invert
#                 cur_pg = []
#             else:
#                 cur_pg.append(vertices[i])
                
#     elif names[count] == 'Milton Keynes North East':
        
#         vertices = []
#         vertices += list(path.vertices[:6])
#         vertices += list(paths[0][count+1].vertices[1:10])
#         vertices += list(path.vertices[15:])
#         codes = [1,]
#         codes += [2] * (len(vertices)-2) 
#         codes.append(79)
        
#         cur_pg = []
#         names[count] = 'Milton Keynes'

#         for i in range(0,len(vertices)):
#             if codes[i] == 79:
#                 pg_x.append([[c[0] for c in cur_pg]])
#                 pg_y.append([[c[1]*-1 for c in cur_pg]]) # multiply by -1 to invert
#                 cur_pg = []
#             else:
#                 cur_pg.append(vertices[i])
    
#     return pg_x, pg_y

# def check_for_holes(xs, ys):
    
#     const_holes = {('1983','1987','1992'):
#                        {'PENRITH AND THE BORDER':'CARLISLE',
#                         'WANSDYKE':'BATH',
#                         'CIRENCESTER AND TEWKESBURY':'CHELTENHAM'},
#                ('1997','2001'):
#                        {'NORTH ESSEX':'COLCHESTER'}
#               }
    
#     for key in const_holes.keys():
        
#         if election in key:
            
#             for const in const_holes[key].keys():
                
#                 count = 0
#                 outer = 0
#                 inner = 0
#                 for name in names:
#                     if name.upper() == const:
#                         outer = count
#                     elif name.upper() == const_holes[key][const]:
#                         inner = count
#                     count += 1

#                 xs[outer][0].append(xs[inner][0][0])
#                 ys[outer][0].append(ys[inner][0][0])
    
#     return xs, ys

# def get_style(elem):
#     style_dict = {}
#     for x in elem.attrib['style'].split(';'):
#         xsplit = x.split(':')
#         style_dict.update({xsplit[0]:xsplit[1]})
#     return style_dict

# def correct_glasgow(names,xs,ys,anomalies):
    
#     names.remove('Glasgow Central')
#     names.remove('path4893')
#     names.append('Glasgow Central')
    
#     vertices = []
#     vertices += list(anomalies['path4893'].vertices[0:11])
#     vertices += list(anomalies['Glasgow Central'].vertices[0:6])
#     vertices += list(anomalies['path4893'].vertices[16:])
#     codes = [1,]
#     codes += [2] * (len(vertices)-2) 
#     codes.append(79)
    
#     cur_pg = []

#     for i in range(0,len(vertices)):
#         if codes[i] == 79:
#             xs.append([[[c[0] for c in cur_pg]]])
#             ys.append([[[c[1]*-1 for c in cur_pg]]]) # multiply by -1 to invert
#             cur_pg = []
#         else:
#             cur_pg.append(vertices[i])
            
#     return names, xs, ys

# election = '1983'
# tree = etree.parse("SVGs/Map1992.svg")
# root = tree.getroot()
# path_elems = root.findall('.//{http://www.w3.org/2000/svg}path')
# names = [elem.attrib['id'] for elem in path_elems]
# paths = parse_svg("SVGs/Map1992.svg")
# #facecolors = [get_style(elem).get('fill', 'none') for elem in path_elems]
# edgecolors = [get_style(elem).get('stroke', 'none') for elem in path_elems]
# linewidths = [get_style(elem).get('stroke_width', 1) for elem in path_elems]

# xs = []
# ys = []
# x_temp = []
# y_temp = []
# count = 0
# colours_to_remove = []
# anomalies = {}

# for path in paths[0]:
#     if 'Milton' in names[count] and election in ('1987','1983'):
#         print(len(xs))
#         x_temp, y_temp = convert_path_into_polygons(path,x_temp,y_temp,count)
#         if names[count] == 'Milton Keynes':
#             xs.append(x_temp)
#             ys.append(y_temp)
#         x_temp = []
#         y_temp = []
#         print(len(xs))
#     elif election in ('1992','1987','1983') and names[count] in ('path4893','Glasgow Central'):
#         anomalies[names[count]] = path
#     else:    
#         x_temp, y_temp = convert_path_into_polygons(path,x_temp,y_temp)
#         if count == len(paths[0])-1 or names[count+1] != names[count]:       
#             xs.append(x_temp)
#             ys.append(y_temp)
#             x_temp = []
#             y_temp = []
#         else:
#             colours_to_remove.append(count)
#     count += 1

# if election in ('1987','1983'):
#     names.remove('Milton Keynes South West')
# if election in ('1992','1987','1983'):
#     names, xs, ys = correct_glasgow(names,xs,ys,anomalies)
    
# xs, ys = check_for_holes(xs,ys)

# # for i in range(len(colours_to_remove)-1,-1,-1):
# #     facecolors.pop(colours_to_remove[i])

# for i in range(len(names)-1,-1,-1):
#     if i != 0 and names[i]==names[i-1]:
#         names.pop(i)
    
# #names = [x.title().replace('_',' ').replace('And','and').replace(' Of ',' of ').replace('Under','under').replace('Weston-Super-Mare','Weston-super-Mare') for x in names]
# facecolors = []
# for name in names:
#     try:
#         if name in CONSTITUENCY.const_list.keys():
#             const_obj = CONSTITUENCY.const_list[name]
#             party = const_obj.election_list[election].party1
#         else:
#             for const in CONSTITUENCY.const_list.values():
#                 if name == const.previous:
#                     const_obj = const
#                     party = const_obj.election_list[election].party1
#         facecolors.append(PARTY.party_colours[party])
#         const_obj = ''
#     except:
#         print(name)
#         0/0

# data=dict(
#     x=xs,
#     y=ys,
#     name=names,
#     colours=facecolors,
#     #results = results
# )

# cds = ColumnDataSource(data)

# TOOLS = "pan,wheel_zoom,reset,hover,save"

# p = figure(
#     title="General Election, " + election, tools=TOOLS,
#     x_axis_location=None, y_axis_location=None,
#     tooltips=[("Name", "@name")],
#     aspect_ratio=0.7)

# p.multi_polygons(xs="x",ys="y",line_width=1,color="colours",line_color='black',name="names",source=cds)

# show(p)

In [250]:
# SVG

# parameters = {'1983':{'shet':574,'scotStart':540,'scotEnd':612},
#               '1987':{'shet':574,'scotStart':540,'scotEnd':612},
#               '1992':{'shet':574,'scotStart':540,'scotEnd':612},}

# def transform_paths(path_obj):
#     a= 0.53385753
#     b = 0
#     c = 0
#     d = 0.5357256
#     e = 7.1570288
#     f = -70.336371
#     new_list = []
#     count = 0
#     for coords in path_obj.vertices:
#         x = a * coords[0] + c * coords[1] + e
#         y = b * coords[0] + d * coords[1] + f
#         path_obj.vertices[count][0] = x
#         path_obj.vertices[count][1] = y
#         count += 1
#     return path_obj

# def transform_shet(path_obj):
#     a= 0.3
#     b = 0
#     c = 0
#     d = 0.3
#     e = 60
#     f = -190
#     new_list = []
#     count = 0
#     for coords in path_obj.vertices:
#         x = a * coords[0] + c * coords[1] + e
#         y = b * coords[0] + d * coords[1] + f
#         path_obj.vertices[count][0] = x
#         path_obj.vertices[count][1] = y
#         count += 1
#     return path_obj

# try:
#     plt.clf()
# except:
#     pass

# #r = requests.get('https://commons.wikimedia.org/wiki/File:UK_General_Election,_1983.svg')
# tree = etree.parse("SVGs/Map1983.svg")
# root = tree.getroot()
# width = int(re.match(r'\d+', root.attrib['width']).group())
# height = int(re.match(r'\d+', root.attrib['height']).group())
# path_elems = root.findall('.//{http://www.w3.org/2000/svg}path')

# paths = []
# count = 0

# names = [elem.attrib['id'] for elem in path_elems]
# scotland = False

# for elem in path_elems:
#     #print(names[count], end='')
#     if elem.attrib['d'] == '':
#         continue
#     if names[count] == 'Roxburgh_and_Berwickshire':
#         scotland = True
#         #pass
#     if names[count] == 'Aberavon':
#         scotland = False
#     #if count >= parameters[election]['scotStart'] and count <= parameters[election]['scotEnd']:
#     if scotland:
#         #if count == parameters[election]['shet']:
#         if names[count] == 'Orkney_and_Shetland':
#             paths.append(transform_shet(parse_path(elem.attrib['d'])))
#         else:
#             paths.append(transform_paths(parse_path(elem.attrib['d'])))
#     else:
#         paths.append(parse_path(elem.attrib['d']))
#     count += 1
    

# facecolors = [get_style(elem).get('fill', 'none') for elem in path_elems]
# edgecolors = [get_style(elem).get('stroke', 'none') for elem in path_elems]
# linewidths = [get_style(elem).get('stroke_width', 1) for elem in path_elems]

# collection = mpl.collections.PathCollection(paths, 
#                                       edgecolors=edgecolors, 
#                                       linewidths=linewidths,
#                                       facecolors=facecolors)

# def update_annot(ind):
#     pos=sc.get_offsets()[ind["ind"][0]]
#     annot.sy=pos
#     text="{},{}".format(" ".join(list(map(str,ind["ind"]))),
#         " ".join([names[n] for n in ind["ind"]]))
#     annot.set_text(text)
#     annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
#     annot.get_bbox_patch().set_alpha(0.4)
    
# def hover(event):
#     vis=annot.get_visible()
#     if event.inaxes == ax:
#         cont, ind = sc.contains(event)
#         if cont:
#             update_annot(ind)
#             annot.set_visible(True)
#             fig.canvas.draw_idle()

# xs = []
# ys = []
# for path in paths:
#     x_p, y_p = convert_path_into_polygons(path)
#     xs.append(x_p)
#     ys.append(y_p)

# # #with plt.ioff():
# # fig = plt.figure(figsize=(12,12))
# # ax = fig.add_subplot(111)
# # collection.set_transform(ax.transData)
# # ax.add_artist(collection)
# # ax.set_xlim([0, width])
# # ax.set_ylim([height, -120])
# # plt.xlim(xmin=-200,xmax=800)
# # ax.axis('off')
# # #ax.imshow(image)
# # annot = ax.annotate("",xy=(0,0),xytext=(20,20),textcoords="offset points",bbox=dict(boxstyle="round",fc="w"),arrowprops=dict(arrowstyle="->"))
# # annot.set_visible(False)            
# # fig.canvas.mpl_connect("motion_notify_event",hover)
# # #disconnect_zoom = zoom_factory(ax)
# # #pan_handler = panhandler(fig)
# # map_output = widgets.Output()
# # map_gui = widgets.VBox([map_output],layout=Layout(width='100%'))
# # display(map_gui)
# # #with map_output:
# # plt.show()


# names = [x.title().replace('_',' ') for x in names]

# data=dict(
#     x=xs,
#     y=ys,
#     name=names,
#     colours=facecolors,
#     #results = results
# )

# cds = ColumnDataSource(data)

# TOOLS = "pan,wheel_zoom,reset,hover,save"

# p = figure(
#     title="General Election, 2019", tools=TOOLS,
#     x_axis_location=None, y_axis_location=None,
#     tooltips=[("Name", "@name")],
#     aspect_ratio=0.5)

# p.multi_polygons(xs="x",ys="y",line_width=1,color="colours",line_color='black',name="names",source=cds)

# # source = ColumnDataSource(dict(xs=xs, ys=ys))

# # plot = Plot(
# #     title=None, width=300, height=300,
# #     min_border=0, toolbar_location=None)

# # glyph = MultiPolygons(xs="xs", ys="ys", line_width=2)
# # plot.add_glyph(source, glyph)

# # xaxis = LinearAxis()
# # plot.add_layout(xaxis, 'below')

# # yaxis = LinearAxis()
# # plot.add_layout(yaxis, 'left')

# # plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
# # plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

# # curdoc().add_root(plot)

# # p.grid.grid_line_color = None
# # p.hover.point_policy = "follow_mouse"

# # patch_renderer  = p.patches('x', 'y', source=cds,
# #           fill_color={"field":"colours"},
# #           fill_alpha=0.7, line_color="white", line_width=0.5)

# show(p)

In [ ]:
# def get_hex(df):
    
#     hex_coords = list(df['2010 Hex'])
#     final_coords = []
#     for hex_string in hex_coords:
#         hex_split = hex_string.split(',')
#         hex_split = [int(x) for x in hex_split]
#         final_coords.append(hex_split)
    
#     # Horizontal cartesian coords
#     hcoord_n = [c[0] for c in final_coords]
#     # Vertical cartersian coords
#     vcoord_n = [2. * np.sin(np.radians(60)) * (c[1] - c[2]) /3. for c in final_coords]
    
#     return hcoord_n, vcoord_n

# def update_annot(ind):
    
#     pos=sc.get_offsets()[ind["ind"][0]]
#     #annot.xy=pos
#     annot.set_x(pos[0])
#     annot.set_y(pos[1])
#     text = names[ind['ind'][0]]
#     info_box_over.value=text
#     annot.set_text(text)
#     annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
#     annot.get_bbox_patch().set_alpha(0.4)
    
# def hex_hover(event):
    
#     global names, sc
#     #outputW.value = str(event) #.xdata) + " " + str(event.ydata)
#     vis=annot.get_visible()
#     if event.inaxes == ax:
#         cont, ind = sc.contains(event)
# #         outputW.value=(str(cont) + " " + str(ind))
#         if cont:
#             #names = list(hex_df['Constituency'])
#             text = names[ind['ind'][0]]
#             #text += str(event)
#             info_box_over.value=text
#             #update_annot(ind)
#             annot.set_visible(True)
#             fig.canvas.draw_idle()


In [89]:
# def switch_election(wid):

#     if map_options.value == 'Map':
#         map_info_box.children = [map_box,info_box]
#         load_map()
#     else:
#         map_info_box.children = [hex_box]
#         display_hex()

# def switch_country(wid):

#     pass

# def display_hex(opw):

#     #hex_output.clear_output()
#     opw.clear_output()

#     #with hex_output:
#     with opw:

#         election = election_option.value
#         names = list(hex_df['Constituency'])
#         colours = get_colours(names,election)
#         bk_xs, bk_ys = get_hex_coords(hex_df)
#         results = [CONSTITUENCY.const_list[x].election_list[election].results.to_dict() for x in names]

#         data=dict(
#             x=bk_xs,
#             y=bk_ys,
#             name=names,
#             colours=colours,
#             results = results
#         )

#         cds = ColumnDataSource(data)

#         TOOLS = "pan,wheel_zoom,reset,hover,save"

#         p = figure(
#             title="General Election, 2019", tools=TOOLS,
#             x_axis_location=None, y_axis_location=None,
#             tooltips=[
#                 ("Name", "@name"), #("Unemployment rate", "@rate%"), ("(Long, Lat)", "($x, $y)")
#             ])

#         p.grid.grid_line_color = None
#         p.hover.point_policy = "follow_mouse"

#         patch_renderer  = p.patches('x', 'y', source=cds,
#                   fill_color={"field":"colours"},
#                   fill_alpha=0.7, line_color="white", line_width=0.5)

#         div = Div(width=400)
#         indicator_div = Div(text="",width=350)
#         layout = bkCol(bkRow(p, indicator_div))

#         tap_tool = TapTool(renderers=[patch_renderer])
#         p.add_tools(tap_tool)

#         patch_indicator_callback = CustomJS(args=dict(cds=cds, div=indicator_div, election=election), code="""
#           div.text = "<h2>" + cds.data['name'][cb_obj.indices] + "</h2><h3>" + election + " General Election Results</h3><br>" + 
#           "<b>Winning Party:</b> " + cds.data['results'][cb_obj.indices]['Party'][0] + "<br><br>" +
#           "<table>" +
#           "<tr>" +
#             "<th>Party</th>" +
#             "<th>Candidate</th>" +
#             "<th>Votes</th>" +
#             "<th>Percent</th>" +
#           "</tr>" +
#           "<tr>"
#           let partyList = cds.data['results'][cb_obj.indices]['Party']
#           for (let i = 0; i < Object.keys(partyList).length; i++) {
#             div.text += "<td>" + cds.data['results'][cb_obj.indices]['Party'][i] + "</td>";
#             div.text += "<td>" + cds.data['results'][cb_obj.indices]['Candidate'][i] + "</td>";
#             div.text += "<td>" + cds.data['results'][cb_obj.indices]['Votes'][i] + "</td>";
#             div.text += "<td>" + cds.data['results'][cb_obj.indices]['Percent'][i] + "</td></tr>";
#             }
#           div.text += "</table>"
#           """)

#         cds.selected.js_on_change('indices', patch_indicator_callback)

#         show(layout)

# title = widgets.HTML("<h2>United Kingdom General Elections</h2>")
# country_option = widgets.ToggleButtons(options=['UK','US','Canada'],
#                                         layout=Layout(margin="10px 0px 20px 0px"),
#                                         button_style='danger')
# election_option = widgets.ToggleButtons(options=['2019','2017','2015','2010'],
#                                 layout=Layout(margin="10px 0px 20px 0px"),
#                                 button_style='warning')
# country_option.observe(switch_country,names='value')
# election_option.observe(switch_election,names='value')
# over_label = widgets.HTML("<b>Constituency at cursor:</b>")
# info_box_over = widgets.Text()
# info_box_click = widgets.Output()
# info_box = widgets.VBox([over_label,info_box_over,info_box_click],layout=Layout(width="40%"))
# map_options = widgets.ToggleButtons(options=['Map','Cartogram'],button_style='info')
# map_options.observe(switch_election,names='value')
# map_output = widgets.VBox()
# hex_output = widgets.Output()
# hex_box = widgets.VBox([map_options,hex_output],layout=Layout(width='100%'))
# m = Map(center=(54, 0), zoom=5, layout=Layout(height="100%",width="100%"),scroll_wheel_zoom=True)
# map_box = widgets.VBox([map_options,m],layout=Layout(width='65%'))
# map_info_box = widgets.HBox([map_box,info_box],layout=Layout(height="1000px",width="100%"))
# gui = widgets.VBox([title,country_option,election_option,map_info_box],layout=Layout(align_items='center'))
# display(gui)

In [14]:
# from bokeh.io import show
# from bokeh.models import LogColorMapper
# from bokeh.palettes import Viridis6 as palette
# from bokeh.plotting import figure
# from bokeh.sampledata.unemployment import data as unemployment
# from bokeh.sampledata.us_counties import data as counties

# palette = tuple(reversed(palette))

# counties = {
#     code: county for code, county in counties.items() if county["state"] == "tx"
# }

# county_xs = [county["lons"] for county in counties.values()]
# county_ys = [county["lats"] for county in counties.values()]

# county_names = [county['name'] for county in counties.values()]
# county_rates = [unemployment[county_id] for county_id in counties]
# color_mapper = LogColorMapper(palette=palette)

# data=dict(
#     x=county_xs,
#     y=county_ys,
#     name=county_names,
#     rate=county_rates,
# )

# TOOLS = "pan,wheel_zoom,reset,hover,save"

# p = figure(
#     title="Texas Unemployment, 2009", tools=TOOLS,
#     x_axis_location=None, y_axis_location=None,
#     tooltips=[
#         ("Name", "@name"), ("Unemployment rate", "@rate%"), ("(Long, Lat)", "($x, $y)")
#     ])
# p.grid.grid_line_color = None
# p.hover.point_policy = "follow_mouse"

# p.patches('x', 'y', source=data,
#           fill_color={'field': 'rate', 'transform': color_mapper},
#           fill_alpha=0.7, line_color="white", line_width=0.5)

# show(p)

In [ ]:
# #Geo file conversion
# from osgeo import gdal

# def shapefile2geojson(infile, outfile, fieldname):
#     '''Translate a shapefile to GEOJSON.'''
#     options = gdal.VectorTranslateOptions(format="GeoJSON", dstSRS="EPSG:4326")
#     gdal.VectorTranslate(outfile, infile, options=options)
#     print("Done")

# gdal.UseExceptions()
# shapefile2geojson('Data/Can/lfed000b21a_e.shp','Data/Can/canED.geojson','')

In [ ]:
# count = 0
# for feature in gj_2005['features']:
#     if feature['properties']['name'] == 'Bristol North West':
#         print(count)
#     count += 1

In [36]:
# # Original data
# import geojson
# with open('Data/GB/gb_const_region.geojson') as f:
#     gj = geojson.load(f)
# with open('Data/NI/2008_Boundaries.geojson') as f:
#     gj_ni = geojson.load(f)

# # Correct GB names
# for feature in gj['features']:
#     feature['properties']['NAME'] = feature['properties']['NAME'].replace(' Boro Const','').replace(' Co Const','').replace(' Burgh Const','').replace('Birmingham,','Birmingham').replace('St.','St')

# # Correct NI names
# for i in gj_ni['features']:
#     i['properties']['PC_NAME'] = i['properties']['PC_NAME'].title()
#     if i['properties']['PC_NAME'] == 'Fermanagh And South Tyrone':
#         i['properties']['PC_NAME'] = 'Fermanagh and South Tyrone'
#     if i['properties']['PC_NAME'] == 'Newry And Armagh':
#         i['properties']['PC_NAME'] = 'Newry and Armagh'

# # Bristol North West cut-off - 80
# North vertex - -2.7022866, 51.5226722
# s_ver = [-2.717705, 51.500679]
# n_ver = [-2.701623, 51.522668]
# bnw_coords = gj_2005['features'][73]['geometry']['coordinates'][0]
# new_coords = []
# stop=True

# for coord in bnw_coords:
#     if coord == s_ver:
#         stop = False
#         new_coords.append(coord)
#     elif coord == n_ver:
#         stop = True
#         new_coords.append(coord)
#     elif not stop:
#         new_coords.append(coord)

# gj_2005['features'][73]['geometry']['coordinates'][0] = new_coords



In [4]:
# # Load election data

# df_2010 = pd.read_excel('UK_Elections.xlsx',sheet_name='2010')
# df_2015 = pd.read_excel('UK_Elections.xlsx',sheet_name='2015')
# df_2017 = pd.read_excel('UK_Elections.xlsx',sheet_name='2017')
# df_2010.fillna('',inplace=True)
# df_2015.fillna('',inplace=True)
# df_2017.fillna('',inplace=True)


In [24]:
# # Add results to geojson

# def add_results(gj,col='Winner',name_field='NAME'):
#     gj['features'][0]['properties']['NAME']
#     for feature in gj['features']:
#         name = feature['properties']['NAME']
#         try:
#             winner10 = df_2010.loc[df_2010[df_2010['Constituency']==name].index[0],col]
#             winner15 = df_2015.loc[df_2015[df_2015['Constituency']==name].index[0],col]
#             winner17 = df_2017.loc[df_2017[df_2017['Constituency']==name].index[0],col]
#             feature['properties']['2010_Winner'] = winner10
#             feature['properties']['2015_Winner'] = winner15
#             feature['properties']['2017_Winner'] = winner17
#         except:
#             print(name)
        
#     return gj

# gj = add_results(gj)
# gj_ni = add_results(gj_ni, name_field='PC_NAME')

In [49]:
# # Write to geojson

# with open('Data/GB/2005_Consts.geojson', 'w') as outfile:
#      geojson.dump(gj_2005, outfile)
    
# with open('Data/GB/ni_with_results.geojson', 'w') as outfile:
#     geojson.dump(gj_ni, outfile)

In [12]:
# count = 0
# names = []
# for feature in gj['features']:
#     feature['properties']['NAME'] = feature['properties']['NAME'].replace(' Boro Const','').replace(' Co Const','').replace(' Burgh Const','').replace('Birmingham,','Birmingham').replace('St.','St')
#     count += 1

In [ ]:
# names = [feature['properties']['NAME'] for feature in gj['features']]
# s = pd.Series(names)
# s.to_excel('UK_Elections_temp.xlsx')

In [6]:

# from xml.dom import minidom
# xmldoc = minidom.parse('UK_General_Election,_1983.svg')
# itemlist = xmldoc.getElementsByTagName('polygon')

# for s in itemlist :
#     for p in s.attributes['points'].value.split():
#         print(p)

# import numpy as np
# import pylab
# for s in itemlist :
#     p = np.vstack(map(lambda x: np.fromstring(x,sep=','), s.attributes['points'].value.split()))
#     p=np.vstack([p,p[0,:]])
#     pylab.plot(p[:,0],p[:,1])
# pylab.show()